<a href="https://colab.research.google.com/github/JonathanPhillips/2022-VOR-Model/blob/main/VOR2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

In [36]:
df['RUSH_YD'] = df['RUSH_YD'].astype(str).astype(float)
df['REC_YD'] = df['REC_YD'].astype(str).astype(float)

In [39]:
df.dtypes

Player       object
Team         object
POS          object
RUSH_ATT    float64
RUSH_YD     float64
RUSH_TD     float64
REC         float64
REC_YD      float64
REC_TD      float64
FL          float64
PASS_ATT    float64
CMP         float64
PASS_YD      object
PASS_TD     float64
INTS        float64
dtype: object

In [42]:
df['PASS_YD'] = df['PASS_YD'].str.replace(',', '').astype(float)

In [44]:
df.dtypes

Player       object
Team         object
POS          object
RUSH_ATT    float64
RUSH_YD     float64
RUSH_TD     float64
REC         float64
REC_YD      float64
REC_TD      float64
FL          float64
PASS_ATT    float64
CMP         float64
PASS_YD     float64
PASS_TD     float64
INTS        float64
dtype: object

In [4]:
df = pd.read_csv('/content/all_compiled.csv', index_col=0)

comma_columns = ['RUSH_YD', 'REC_YD', 'PASS_YD']
for column in comma_columns:
  df[column] = df[column].str.replace(',', '').astype(float)

df['PPR'] = df['RUSH_YD'] + df['REC_YD'] + df['PASS_TD']*.04 + df['RUSH_TD'] + df['REC_TD']*6 \
+ df['PASS_TD']*4 + df['REC'] + df['FL']*-2 + df['INTS']*-2

df.sort_values(by='PPR', ascending=False).head()

,Player,Team,POS,RUSH_ATT,RUSH_YD,RUSH_TD,REC,REC_YD,REC_TD,FL,PASS_ATT,CMP,PASS_YD,PASS_TD,INTS,PPR
1,Jonathan Taylor,IND,RB,323.9,1618.7,13.6,42.6,363.6,1.7,2.7,0.0,0.0,0.0,0.0,0.0,2043.3
3,Derrick Henry,TEN,RB,377.7,1572.4,11.6,30.3,227.3,0.4,2.6,0.0,0.0,0.0,0.0,0.0,1838.8
1,Cooper Kupp,LAR,WR,4.1,20.8,0.1,123.7,1569.3,11.9,0.8,0.0,0.0,0.0,0.0,0.0,1783.7
2,Austin Ekeler,LAC,RB,220.2,945.2,9.4,74.5,687.4,5.6,2.3,0.0,0.0,0.0,0.0,0.0,1745.5
4,Dalvin Cook,MIN,RB,295.3,1271.9,9.9,47.9,374.9,1.1,2.8,0.0,0.0,0.0,0.0,0.0,1705.6


In [14]:
ADP_URL = 'https://www.fantasypros.com/nfl/adp/ppr-overall.php'

res = requests.get(ADP_URL)

soup = BS(res.content, 'html.parser')

table = soup.find('table', attrs={
    'id': 'data'
})

adp_df = pd.read_html(str(table))[0]
adp_df['Player'] = adp_df['Player Team (Bye)'].apply(lambda x: ' '.join(x.split()[:-2]))
adp_df['Team'] = adp_df['Player Team (Bye)'].apply(lambda x: x.split()[-2])
adp_df.head(50)

,Rank,Player Team (Bye),POS,ESPN,RTSports,MFL,Fantrax,FFC,Sleeper,AVG,Player,Team
0,1,Christian McCaffrey CAR (13) O,RB1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Christian McCaffrey CAR,(13)
1,2,Dalvin Cook MIN (7),RB2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Dalvin Cook,MIN
2,3,Alvin Kamara NO (6),RB3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,Alvin Kamara,NO
3,4,Derrick Henry TEN (13),RB4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,Derrick Henry,TEN
4,5,Ezekiel Elliott DAL (7),RB5,8.0,5.0,6.0,5.0,5.0,5.0,5.8,Ezekiel Elliott,DAL
5,6,Davante Adams LV (8),WR1,5.0,6.0,5.0,6.0,6.0,10.0,6.6,Davante Adams,LV
6,7,Travis Kelce KC (12),TE1,7.0,9.0,10.0,8.0,8.0,12.0,8.4,Travis Kelce,KC
7,8,Aaron Jones GB (13),RB6,10.0,11.0,7.0,10.0,9.0,11.0,9.4,Aaron Jones,GB
8,9,Saquon Barkley NYG (10),RB7,6.0,10.0,11.0,11.0,13.0,7.0,9.6,Saquon Barkley,NYG
9,10,Austin Ekeler LAC (7),RB8,13.0,8.0,12.0,12.0,7.0,13.0,10.4,Austin Ekeler,LAC


In [15]:
adp_df['POS'] = adp_df['POS'].str[:2]
adp_df = adp_df.loc[:, ['Player', 'Team', 'POS', 'Rank']]
adp_df.head()

,Player,Team,POS,Rank
0,Christian McCaffrey CAR,(13),RB,1
1,Dalvin Cook,MIN,RB,2
2,Alvin Kamara,NO,RB,3
3,Derrick Henry,TEN,RB,4
4,Ezekiel Elliott,DAL,RB,5


In [17]:
adp_cutoff = 150
adp_cutoff_df = adp_df.sort_values(by='Rank')[:adp_cutoff]
adp_cutoff_df = adp_cutoff_df.merge(df.loc[:, ['Player', 'Team', 'POS', 'PPR']], on=['Player', 'Team', 'POS'])
adp_cutoff_df.head(50)

,Player,Team,POS,Rank,PPR
0,Dalvin Cook,MIN,RB,2,1705.600
1,Alvin Kamara,NO,RB,3,1513.100
2,Derrick Henry,TEN,RB,4,1838.800
3,Ezekiel Elliott,DAL,RB,5,1149.300
4,Davante Adams,LV,WR,6,1509.400
5,Travis Kelce,KC,TE,7,1385.500
6,Aaron Jones,GB,RB,8,1350.600
7,Saquon Barkley,NYG,RB,9,1302.800
8,Austin Ekeler,LAC,RB,10,1745.500
9,Jonathan Taylor,IND,RB,11,2043.300


In [19]:
replacement_values = {}
for _, row in adp_cutoff_df.iterrows():
  replacement_values[row['POS']] = row['PPR']

replacement_values

{'QB': 553.932, 'RB': 923.0, 'TE': 625.3, 'WR': 940.0999999999999}

In [24]:
vor_df = df.loc[:, ['Player', 'Team', 'POS', 'PPR']].rename({'POS': 'Position'}, axis=1)
vor_df['VOR'] = vor_df.apply(lambda row: row['PPR'] - replacement_values[row['Position']], axis=1)
vor_df.head()

,Player,Team,Position,PPR,VOR
1,Jonathan Taylor,IND,RB,2043.3,1120.3
2,Austin Ekeler,LAC,RB,1745.5,822.5
3,Derrick Henry,TEN,RB,1838.8,915.8
4,Dalvin Cook,MIN,RB,1705.6,782.6
5,Joe Mixon,CIN,RB,1493.1,570.1


In [25]:
vor_df.sort_values(by='VOR', ascending=False).head(20)

,Player,Team,Position,PPR,VOR
1,Jonathan Taylor,IND,RB,2043.3,1120.3
3,Derrick Henry,TEN,RB,1838.8,915.8
1,Cooper Kupp,LAR,WR,1783.7,843.6
2,Austin Ekeler,LAC,RB,1745.5,822.5
4,Dalvin Cook,MIN,RB,1705.6,782.6
1,Travis Kelce,KC,TE,1385.5,760.2
8,Christian McCaffrey,CAR,RB,1665.2,742.2
3,Justin Jefferson,MIN,WR,1680.3,740.2
2,Deebo Samuel,SF,WR,1676.6,736.5
7,Najee Harris,PIT,RB,1594.1,671.1
